# Coherent info

In [1]:
import numpy as np
from collections import Counter
from typing import Tuple, Iterable, Dict, List
from coherentinfo.moebius import build_moebius_code_vertex
from coherentinfo.linalg import finite_field_gauss_jordan_elimination

In [11]:
# -------------------------------
# Möbius/cylinder strip (d=2)
# -------------------------------
def build_mobius_code(length: int, width: int) -> Tuple[np.ndarray, np.ndarray, int]:
    assert length >= 2 and width >= 2

    num_h_edges = length * (width - 1)  # horizontal edges
    num_v_edges = length * width  # vertical edges
    num_edges = num_h_edges + num_v_edges

    def idx_h(y, x):
        return y * length + (x % length)

    def idx_v(y, x):
        return num_h_edges + y * length + (x % length)

    # Z-check matrix
    rows = []
    for y in range(width - 1):
        for x in range(length):
            row = np.zeros(num_edges, dtype=np.uint8)
            row[idx_h(y, x)] ^= 1
            row[idx_h(y, x - 1)] ^= 1
            row[idx_v(y, x)] ^= 1
            row[idx_v(y + 1, x)] ^= 1
            rows.append(row)
    h_z = np.array(rows, dtype=np.uint8)

    # Logical Z along vertical edges in second row
    l_z = np.zeros(num_edges, dtype=np.uint8)
    y0 = width // 2
    for x in range(length):
        l_z[idx_v(y0, x)] = 1

    return h_z, l_z, num_edges


def build_mobius_code_dual(L: int, w: int) -> Tuple[np.ndarray, np.ndarray, int]:
    """
    Build H_X (plaquette checks) and l_X (vertical logical X) for a d=2 strip
    (periodic in x, open in y) with horizontal edges = L*(w-1), vertical edges = L*w.
    """
    nH = L * (w - 1)  # horizontal edges
    nV = L * w  # vertical edges
    n = nH + nV

    def idxH(y, x):
        return y * L + x  # horizontal edges indexed by y (row), x (column)

    def idxV(y, x):
        return nH + y * L + x  # vertical edges indexed by y (row), x (column)

    rows = []
    for y in range(w):
        for x in range(L):
            row = np.zeros(n, dtype=np.uint8)

            if y == 0:  # top row
                row[idxH(x, 0) if L == 1 else x] = 1  # top horizontal
                row[idxV(y, x)] = 1  # left vertical
                row[idxV(y, (x + 1) % L)] = 1  # right vertical
            elif y == w - 1:  # bottom row
                row[idxH(y - 1, x)] = 1  # bottom horizontal
                row[idxV(y, x)] = 1  # left vertical
                row[idxV(y, (x + 1) % L)] = 1  # right vertical
            else:  # middle row
                row[idxH(y - 1, x)] = 1  # top horizontal
                row[idxH(y, x)] = 1  # bottom horizontal
                row[idxV(y, x)] = 1  # left vertical
                row[idxV(y, (x + 1) % L)] = 1  # right vertical
            rows.append(row)

    H_X = np.array(rows, dtype=np.uint8)

    # Logical X along first vertical column
    l_X = np.zeros(n, dtype=np.uint8)
    for y in range(w):
        l_X[idxV(y, 0)] = 1

    return H_X, l_X, n

In [16]:
width = 4
length = 2
h_z, l_z, num_edges = build_mobius_code(length, width)

In [17]:
h_z @ l_z

array([0, 0, 1, 1, 1, 1], dtype=uint8)

In [7]:
pippo, pluto = h_z.shape

In [8]:
pippo

15

In [9]:
pluto

35

In [18]:
a

0